In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import fiona
import pydash
import shapely.geometry
from shapely.geometry import Polygon, Point, shape
import pyproj
import numpy as np
from affine import Affine
from rasterio.features import rasterize

In [ ]:
from kk.dggs import DGGS
import kk.tools

dg = DGGS()

In [ ]:
poly, src_crs = kk.tools.load_polygons('/Users/kirill/wk/dggs/data/act_aea.shp')
poly = poly[0]
display(poly, src_crs)

In [ ]:
scale_level = 12#13

overlaps = dg.compute_overlap(scale_level, *poly.boundary.xy, crs=src_crs)

if len(overlaps) != 1:
    print('Warning polygon crosses top-level DGGS regions, this is not supported yet')
    

addr, w, h  = overlaps[0]
tr, *_ = dg.pixel_coord_transform(addr, w, h, native=True, no_offset=True)

display( (addr, w, h) )

poly_m = shapely.ops.transform(dg.to_native(src_crs), poly)
display(poly_m)

poly_pix = shapely.ops.transform(lambda x,y: tr.inverse(x,y), poly_m)
display(poly_pix)

[int(b) for b in poly_pix.bounds]

In [ ]:
im = rasterize([poly_pix], out_shape=(h,w))

plt.imshow(im)
plt.colorbar()